<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Analysis-of-RSS-Feed-Content-with-Topic-Identification" data-toc-modified-id="Analysis-of-RSS-Feed-Content-with-Topic-Identification-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Analysis of RSS-Feed Content with Topic Identification</a></span><ul class="toc-item"><li><span><a href="#Initialization-and-Parametrization" data-toc-modified-id="Initialization-and-Parametrization-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Initialization and Parametrization</a></span></li><li><span><a href="#Data-Collection" data-toc-modified-id="Data-Collection-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Data Collection</a></span><ul class="toc-item"><li><span><a href="#Operation-as-RSS-Aggregator" data-toc-modified-id="Operation-as-RSS-Aggregator-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Operation as RSS Aggregator</a></span></li><li><span><a href="#Loading-Previously-Pickled-RSS-Feeds" data-toc-modified-id="Loading-Previously-Pickled-RSS-Feeds-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Loading Previously Pickled RSS Feeds</a></span></li></ul></li><li><span><a href="#Overview-of-Feed-Data" data-toc-modified-id="Overview-of-Feed-Data-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Overview of Feed Data</a></span><ul class="toc-item"><li><span><a href="#Article-Sizes" data-toc-modified-id="Article-Sizes-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Article Sizes</a></span></li><li><span><a href="#Overview-of-Article-Authors" data-toc-modified-id="Overview-of-Article-Authors-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Overview of Article Authors</a></span></li><li><span><a href="#Tags-Specified-by-the-Feed-Publishers" data-toc-modified-id="Tags-Specified-by-the-Feed-Publishers-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Tags Specified by the Feed Publishers</a></span></li></ul></li><li><span><a href="#Identifying-Topics-within-the-Articles" data-toc-modified-id="Identifying-Topics-within-the-Articles-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Identifying Topics within the Articles</a></span></li><li><span><a href="#Using-Soft-Cosine-Similarity-to-Cluster-Stories" data-toc-modified-id="Using-Soft-Cosine-Similarity-to-Cluster-Stories-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Using Soft Cosine Similarity to Cluster Stories</a></span><ul class="toc-item"><li><span><a href="#LDA-Analysis-with-the-pyLDAvis-Visualization" data-toc-modified-id="LDA-Analysis-with-the-pyLDAvis-Visualization-1.5.1"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>LDA Analysis with the pyLDAvis Visualization</a></span></li></ul></li><li><span><a href="#Sentiment-Analysis" data-toc-modified-id="Sentiment-Analysis-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Sentiment Analysis</a></span></li></ul></li><li><span><a href="#Content-Viewer" data-toc-modified-id="Content-Viewer-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Content Viewer</a></span><ul class="toc-item"><li><span><a href="#A-Technical-Correction" data-toc-modified-id="A-Technical-Correction-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>A Technical Correction</a></span></li></ul></li></ul></div>

# Analysis of RSS-Feed Content with Topic Identification

## Initialization and Parametrization

Initially the functions for data-scraping, loading and summarizing are loaded

In [61]:
%load_ext autoreload
%autoreload
import importlib
%aimport reader
from reader import (loadAllFeedsFromFile, summarizeItems,getFeedDict, 
                    collectArticles, summarizeByDate, summarizeFeeds, smallDict)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


The runtime parameters for all the subsequent cells are pre-configured in the `runParams` dictionary.

The user can specify in the following cell global parameters which will be used in running the scripts in this notebook. Typical usage and variants on the default parameters are hinted at in the commented code. It is also possible to override these and other call parameters in the in notebook-cells themselves.

In [60]:
runParams={'allFeeds':       getFeedDict(),   # A collection of 50 URLs for RSS Feeds, where the names can be chosen freely
                                              # example: {'Buzzfeed': 'https://www.buzzfeed.com/world.xml', 
                                              #           'Al Jazeera': 'http://www.aljazeera.com/xml/rss/all.xml'}
           'collectFeeds':   True,            # Set to True to do more data-scraping
           'inputDir':       './data',        # Directory containing pre-collected RSS-Feed data in pickle format
           'articleLimit':   None,            # Optional specification limiting number of articles to process (for test
                                              # purposes only)
           'numAuthDispl':   30,              # specify limit of number of authors to display/plot
           'numTagsDispl':   30,              # specify limit of number of tags to display/plot
           'numTopics':      30,              # specify limit of number of topics to be found via TFIDF Vectorization
           'numLDATopics':   30,              # specify limit of number of topics to be found via Latent Dirichlet Allocation 
           'numTopicsDispl': 30,              # specify limit of number of topics to display/plot
           'removeTopics':   True,            # specify if articles now referenced by topics should be removed (saves 
                                              # computation time later)
           'ngramRange':     (3,3),           # define min and max ngrams as tuple for deriving topic maps 
           'weModel':        'glove-wiki-gigaword-50', # Word Embedding model for soft cosine similarity calculation
                                              # options are e.g. glove-wiki-gigaword-50,fasttext-wiki-news-subwords-300
                                              # CAUTION: although higher dimension models produce more nuanced results, load
                                              # times are VERY long. In particular the first time use requires a download of
                                              # massive amounts of data which can easily take as much as an hour
           'thresholdFuzzy': 60,              # define cut-off threshold for fuzzy (Levenshtein Distance comparison) in percent
           'thresholdCosine':0.3,             # define cut-off threshold for calculating soft cosine similarity as fraction (0.0 to 1.0)
           'thresholdCosDispl':0.6,           # define cut-off threshold for displaying soft cosine similarity as fraction (0.0 to 1.0)
           'fuzzyDocLimit':  None,            # optional integer limit to number of articles used for fuzzy relevance of
                                              # topics (only for test purposes: default None) 
           'matrixDir':      './outdata',     # relative path for saving matrices to file
           'saveMatrix':     False}           # whether to save matrix to file or not

## Data Collection

Data is collected from a wide range of RSS-Feeds..

A large number of names and URLs of compatible feeds are pre-defined in the convenience function `getFeedDict`. These feeds have been pre-configured for the usage in this project. For example CNN only supplied links to the stories. With more elaborate techniques one could go to the corresponding article site and scrape the site for the content. Due to time limitations and enough material provided CNN was not included. 

The user may however supply his/her own dictionary of feeds for subsequent processing. In the next field an overview is given of the feeds that will used in this project. 

In [ ]:
summarizeFeeds(runParams['allFeeds'])

Data collection from the configured RSS-feeds is invoked in the following cell.
For the purposes of reducing running time or for concentration on a particular group of feeds, the analyses can continue with a smaller subset or even a completely modified set of feed data. 

If the analysis is being repeated and no new data is required, the next cell can be skipped (see rumParam `allFeed`).

In [ ]:
%%time
if runParams['collectFeeds']:
    collectArticles(runParams['allFeeds']) 

### Operation as RSS Aggregator

The collected feeds are automatically stored in pickle files, which can be accumulated over time or discarded if unneeded. This method was used as a rudimentary RSS aggregator for a period of weeks for collection of RSS articles, a technique necessary, since many RSS feeds only publicize a very small number of articles at any given time. 

In this project the collection was simply done manually without any automated collection techniques, which would be appropriate in a larger project. Pickle was able to retain the structure of the data, which worked better than the attempts made using JSON for serialization. Pickle, being a binary format, has the advantage of being more efficient (faster load times, smaller data footprint) and the disadvantage of not being human-readable.   

### Loading Previously Pickled RSS Feeds

The function `loadAllFeedsFromFile` simply loads all previously pickled dictionaries from the configured directory (parameter `inputDir`). For smaller test runs the user may simply remove some of the larger test files from this directory. A consistent test run is possible even with quite small amounts of data, although the significance of the results will of course be diminished.

Some data cleaning occurs to remove some articles which are known to not contain single stories, but rather collections of all the day's news. Such data would diminish the effectiveness of any automatic clustering algorithm. Notably the New York Times daily briefings are removed, as they skew topics/clustering whilst having almost no content.

In [ ]:
allDict=loadAllFeedsFromFile(path = runParams['inputDir'], limitsize=runParams['articleLimit'])

## Overview of Feed Data

Let's take a look at the data we have. Below the source, the title and the content has been listed. In subsequent analyses the title is combined with the content in order to provide more complete data for the analyses. The RSS news articles don't all have content but do supply a significant summary which is utilised here. The articles are cleaned from HTML-tags and saved to item with the key `collatedContents` in each entry. The is great variation in article sizes, an affect that can be viewed in the box plot below. 

In [ ]:
summarizeItems(allDict)

### Article Sizes

Let's take a look at the sizes of the articles over time and from the various feeds. As can be viewed in the box plot the article size varies greatly with a lot of articles only up to a 1000 words. With a large amount of collected articles being summaries this finding is not surprising. To ensure that the article size does not provide too much weight to longer articles, the cosine similarity method is used to calculate document similatiy.  The calculation derives vector representations of the words (or n-grams) in the content, which are then compared in terms of relative angle to each other in a high-dimensional space. The smaller the angle, the higher the similarity. Magnitude (Euclidean distance) is completely discounted by theis method of topic identification. 

Articles in the supplied test corpus were collected from the 2nd of May 2020 until the 31st. Older articles predating the start of the project were removed, this especially because some sources date back a lot further than others. Since the data is from news feeds, older articles are not always available as a characteristic of news feeds is being up to date.

In [ ]:
%%time
summarizeByDate(allDict)

### Overview of Article Authors

For the purpose of plotting statistical information about the articles the Python library [Seaborn](https://seaborn.pydata.org/) was used, also in order to create a certain uniformity in displaying plots.

In [ ]:
#importlib.import_module("samples.seabornScatterPlots") # TODO move to base directory, not samples
%aimport scatterplots 
#importlib.reload(scatterplots)
from scatterplots import displayTopics,displayAuthors,displayTags

In the data that was analysed there have been a significant amount of articles not attributed to authors. As can be viewed in the plot below there are a significant amount of articles that have as an author more general descriptions, such as "admin" or "author". 

Often news sources also refer directly to news agencies being authors of articles like "Associated Press", "Agence France-Presse", "Reuters" or even the Russian News Network "RN", indicating a higher transparency where their news is from, but with no indication as to which person actually wrote the story.

Some feeds have their own notation, which, due to time constraint and importance of this issue when interpreting articles, were not converted most notably, "&#124;" which consists of the "&#" and "124" with ";" resulting in the mark up language "EP&#124;NE" meaning that the article has the two authors EP and NE (who the people behind the abbreviations are is not disclosed).

In [ ]:
displayAuthors(dict=allDict, displayAmount=runParams['numAuthDispl'])

### Tags Specified by the Feed Publishers

Below two seaborn plots are displayed concerning the tags that were used by the publishers of the news. Tags are frequently used when people are searching for news articles and to organise content on a (news) web-site. They tend to be more general in nature and should be able to be provide a source in order to group articles in a general sense. As displayed below there are however still a few different descriptions of tags referring to the same topics. This is because there is no uniformity of tag names across the spectrum of news sources. 

A prime example of how different tags refer to the same talking point is the corona virus pandemic, tags "COVID-19", "Covid-19 Pandemic", "Coronavirus outbreak" and "Coronavirus" all refer to the same topic.

It also has to be noted here as the articles were collected in the timespan of the 2nd to the 31st of May 2020 that the world has been gripped by the corona virus pandemic. As such the tag usage in the feeds in the plot below shows a very big bubble denoting the almost unprecendented importance of the virus. It also has to be mentioned that the pandemic has influenced wide varieties of news and covid is mentioned a lot in combination with other tags (even sport, culture etc..), further increasing its usage.

Some news providers also use tags more in general such as The Guardian or New Europe tending to employ many more tags for differentiation than, for instance, Daily CN News.

In the code below it has been specified to show the `numTagsDispl` most common tags. This is a parameter that can be adjusted to display more or less by the reader. Additionally, the data refers to ```allDict```, which is the dictionary of collected articles that are used in the analysis of this project. This of course can be changed to for instance a custom-dictionary provided by the reader.

From allDict how many of the specified tags occur is calculated for each FeedItem of the named feeds in feedTagMatrix, summing them in the x.y position (Feed, Tagname) in the given matrix below. Specify more output using the `displayAmount` parameter, if you wish.

In [ ]:
%aimport scatterplots
from scatterplots import displayTags
displayTags(allDict, displayAmount=runParams['numTagsDispl'])

## Identifying Topics within the Articles

In [ ]:
%aimport topicmap
#importlib.reload(topicmap)
from topicmap import (displayTopicsAndFeeds,getDocList, getAllTopics,getCustomStopWords, deriveTopicMaps, 
                      updateDictionaryByFuzzyRelevanceofTopics, doStandardInitialize,getAllFeedTopics)

In the next piece of code the docContentList is defined by the `allDict` (i.e. the collected articles held in a dictionary) and the `stop_list`. Both of these parameters can be modified for individual purposes. The `stop_list` refers to an individual list of custom stop words referring to common expressions (words and phrases) that are intrinsically "empty" and need to be ignored for purposes of conent analysis. The pre-configured values are based on the `nltk.corpus` stoplist for English. NLTK ([Natural Language Toolkit](https://www.nltk.org/)) is another Python library that was used in the analysis for tokenization, stemming, tagging, parsing and semantic reasoning. An example of a project-specific stop phrase that was manually added to the base-list taken from NLTK would be "view entire post", which is not related to the content and occurs often enough that it had to be specifically ignored in the analysed content. The interested user may wish to supply other stop word lists or extend the stop word lists used in this project, which are available from the function `getCustomStopWords()`.

Stop words are removed from the content, TF-IDF vectorization with Lemmatization is used to convert the amassed content of the corpus into documents to a matrix of TF-IDF features deriving the specified number of topics. TF-IDF Transformer additionally scales down the impact of tokens that occur very frequently in the given corpus and that are hence empirically less informative than features occuring in a small fraction of the corpus. The user can specify not only the preferred number of features but also the n-gram range

A very interesting parameter to experiment with is the `ngram_range`. The lower and upper bounds allow/restrict to topics with the number of words within the range to be derived from the corpus. With an n-gram range of 1,1 only topics based on single key-words are found. Tests have shown this to produce large numbers of hits with rather mundane every-day words (perhaps the most mundane ones could be used to enrich the stop-word list?). At the other extreme, tests with a range such as 7,8 seem to detect very tabloidesque stories, which raise the suspicion of been produced through syndication from a singe source. The most promising results were obtained with an ngram range of 3,3 or 3,4.

> **Note for future work** 
> It would seem a very good idea to unify similar but not identical topics to a single, most promising topic before proceeding with subsequent processing steps. This could in fact be achieved easily with the use of Levenshtein Distance comparison, as employed for other purposes below.

In [ ]:
%%time
docContentList=getDocList(allDict, 
                          reloaddocs=False,
                          stop_list=getCustomStopWords())
topics= deriveTopicMaps(docContentList, 
                        maxNum=runParams['numTopics'], 
                        ngram_range=runParams['ngramRange'])

The Function `updateDictionaryByFuzzyRelevanceofTopics` adds a list of topic/weighting tuples to each entry of the given `allDict` for each topic that has a fuzzy relevance (the Levenshtein Distance is calculated between the topic phrase and the content of each RSS-entry) of greater than the specified threshold, which can be specified here, as required.
The fuzzy string comparison, that was applied here, analyzes similar instances of topics thus reducing the amount of topics that are analysed and ultimately displayed below. 

For the purpose of the Levenshtein Distance comparison the python package "fuzzywuzzy.python" (https://github.com/seatgeek/fuzzywuzzy) was used.

In [ ]:
%%time
updateDictionaryByFuzzyRelevanceofTopics(topics,
                                         allDict, 
                                         limit=runParams['fuzzyDocLimit'], 
                                         threshold=runParams['thresholdFuzzy'],
                                         remove=runParams['removeTopics'])
displayTopicsAndFeeds(allDict, 
                      numTopics=runParams['numTopicsDispl'])

The scatter plot up above describes the distribution of detected topics for each feed. As is evident from the above, the topics "new coronavirus cases" and "coronavirus latest news" refer to the corona virus pandemic. As was already detected in the tags in a previous section, the corona virus has overwhelmingly dominated the news in the analysed month of May 2020. Particularly classical news feeds of BBC News and The Independent, which update their feeds often as opposed to Der Spiegel that usually posts more editorial pieces, post about the latest updates regarding the pandemic.   

In [ ]:
displayTopics(topics)

Clearly the overwhelmingly most prevalent topic (in May 2020) is the coronavirus pandemic (Covid-19). Not only as the main story of the day (and of the month and year), but also spurning dozens of other sub-stories emanating from the Covid-19 crisis. World War II is also mentioned as part of the topics, as this year marks the 75 anniversary on May 7 that Germany had signed the unconditional surrender marking the end of the war. Kim Jong Un was also derived as a separate topic since he missed his grandfather's birthday celebration (April 15th) and had not been seen in public for an extended period of time and has then reappeared in May 2020. These events have to be accounted for when analysing the the most recent news feed articles.  

## Using Soft Cosine Similarity to Cluster Stories

In the 3D scatter plot map below, a representation of the similarity matrix is displayed. 

The topics are then mapped and a list of topics is added to each entry of the given allEntryDict for each topic that has an LDA fuzzy relevance (see fuzzywuzzy process) of greater than the specified threshold.

The Soft Cosine-Similarity matrix is then calculated with a Word Embedding model that may be specified in runtime parameter `weModel`. Tests have been conducted with fasttext_model300 (dimension 300) or alternatively GloVe (dimension 50). The first call for any word embedding model automatically includes a significant download of model data. For fasttext_model300 this is ca. 1GB of data, which takes roughly 30 minutes. Subsequent calls load this from disk, which also takes 2-3 minutes. Alternatively, the first call for "GloVe (w2v_model)" includes a download of ca. 60MB of data, which takes ca. 3 minutes, subsequent calls load this from disk which also takes 30 seconds. Thus, time for download/load should be considered when deciding to calculate the SoftCosine-Similarity matrix with a dimension of 300 or 50. 

The matrix is subsequently saved to file (if `saveMatrix`is specified as `True`). A spectral analysis and dimension reduction (using the PCA method) is applied to the similarity matrix. The colourization is taken from the results of the spectral clustering, a representation of the coordinates of the "bubbles" is achieved through dimension reduction using the PCA method, to create an optical separation according to the cosine similarity. The user may specify the number of topics to display and the cut-off threshold for deciding if an article should be represented in the plot. The matrix is then presented using plotScatter3D with tool tips.

The tooltip is an HTML string for displaying the RSS-entry infos, currently title, feedname, date of publication and author. What is displayed can be adjusted in the code according to what may be required.

The sizes of the spheres in the plot are scaled in proportion to the (highest) degree of cosine similarity for the article in relation to the cluster, i.e. larger spheres indicate a higher topical relevance for that article.

> **Note for future work** 
> Different methods of dimension reduction are also available, such as TruncateSVD and/or TSNE, which need to be tested additionally and potentially in combination with each other to improve the quality of optical separation. 

In [ ]:
%aimport similarity
importlib.reload(similarity)
from similarity import deriveSoftCosineSimilarityMatrix,do3DPlotOfCosineSimilarity ,showPyLDAvis, saveDFPickle, loadMatrixFiles

In [ ]:
%%time
matrix=deriveSoftCosineSimilarityMatrix(allDict, weName=runParams['weModel'], simThreshold=runParams['thresholdCosine'])
if (runParams['saveMatrix']):
    saveDFPickle(matrix, path=runParams['matrixDir']) 
    
# matrix can be reloaded later, if needed, using 
# matrixList=loadMatrixFiles(path=runParams['matrixDir']) <-- returns list

Now that the similarity matrix has been calculated it is very quick and easy to produce additional plots. For instance with a higher threshold, thus filtering out less significant articles or specifying a different number of topics to color-code.

In [ ]:
do3DPlotOfCosineSimilarity(allDict,matrix=matrix,numTopics=runParams['numTopics'], threshold=runParams['thresholdCosDispl'])

And now here a further example with freely specified topic number and threshold:

In [ ]:
do3DPlotOfCosineSimilarity(allDict,matrix=matrix,numTopics=20, threshold=0.5)

### LDA Analysis with the pyLDAvis Visualization

As was explained previously, there are different techniques available to generate the topics to be displayed. One option that is touched upon here is LDA or the Latent Dirichlet Algorithm. LDA accounts for the interchangeability of words and strings. LDAvis, as used below, is a technique of interactively visualising multiple dimensions. The package is imported from https://pyldavis.readthedocs.io/en/latest/index.html.

The data preparation involves: 
* creating a dictionary of unique IDs for all contents of our corpus
* converting each and every sentence into bag-of-words vectors
* creating an LDA model with the specified number of topics (parametrizable with `numLDATopics`) using the Gensim LDA-Modellibrary

The pyLDAvis widget is then able to visualize the combined results, alowing for interactive usage.

The most relevant terms are displayed below. Moving the cursor over one of the terms shows the conditional distribution of the topic. Again the reader is able to parametrically specify the `numTopics` that are used. 
> **Note for future work** pyLDAvis only seems to support unigram analysis. It is thought that this may be extended to n-gram analysis through a modified usage of the LDA model creation employed here.

In [ ]:
%%time
#importlib.reload(reader)
#importlib.reload(similarity)
from similarity import showPyLDAvisNB
showPyLDAvisNB(allDict,numTopics=runParams['numLDATopics'])

The red bars in the interactive graphic above represent the estimated number of times a given term was generated by a given topic. It is almost impossible to find out what the context of the terms is, as the the terms used are single words thus bi-grams and other n-grams would provide a better overview of what exactly is meant (see "Note for future work" above).
The blue bars represent the overall frequency of the terms in each topic.

Thus, the term "us" is mentioned close to 400 times and when for instance topic number 8 is selected the estimated number of times a "us" was generated by topic 8 is about the 100 mark.  

The topic-term circles are proportional in size to the frequency with which a term is estimated to have been generated by the topics. Evidently topic-term
circles related to "health" are among of the largest circles generated. When a term is selected on the right hand side, the circle area changes in area to the
topic-term relative for that term. For example by hovering over the term "digital" the topic-term circle number 8 increase markedly.   

## Sentiment Analysis

A sentiment analysis of each article is conducted using [Vader](https://pypi.org/project/vader-sentiment/).A 3d scatterplot of the sentiment for topically grouped sets of articles is displayed. The coordinates of the spheres are derived from the positive, negative and neutral sentiment values. The colour represents the compound sentiment. The size of the bubbles are exponentially proportional to the strength of topicality for the preferred topic (the  topical relevance percentage having been calculated and persisted on invoking the `updateDictionaryByFuzzyRelevanceofTopics` function). The user may cycle through all topics, which can be conveniently chosen from the pull-down list above the widget. 

A change of topic triggers a redraw of the 3d-visualization with the sentiment reptresentation of the articles. Further information, identifying each article on display, is displayed per tooltip on hovering with the mouse over each sphere.

In [ ]:
%aimport sentiment
importlib.reload(sentiment)
from sentiment import displaySentiment3D
displaySentiment3D(allDict, topics)

# Content Viewer

With the following tool you can browse the contents of the corpus. Specifying a topic, all RSS-entries with even a minimal reference to that topic may be selected from the drop down list.

In [ ]:
%aimport sentiment
importlib.reload(sentiment)
from sentiment import contentsViewer
contentsViewer(allDict, topics)

## A Technical Correction

Here a small hack to correct the width of the cells which seem to get disturbed by the pyLDAvis widget

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))